In [1]:
from __future__ import print_function 
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual 
from IPython.core.display import display, HTML

import pandas as pd
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sb
import squarify 

%matplotlib inline

# Analysis of COVID-19 outbreak cases

In [2]:
#Data Collection
# Retrieving the LIVE COVID19 Stats updated by John Hopkins University in github
# Loading data right from the source:
confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
death = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [3]:
#Data Preparation
#confirmed.columns
confirmeddrop = confirmed.drop([ 'Lat', 'Long'], axis =1)
deathdrop = death.drop([ 'Lat', 'Long'], axis =1)
recovereddrop = recovered.drop([ 'Lat', 'Long'], axis =1)
#recovereddrop

In [4]:
confirmed_drop_cols = confirmeddrop.drop(['Province/State'],axis = 1)
death_drop_cols = deathdrop.drop(['Province/State'],axis = 1)
recovered_drop_cols = recovereddrop.drop(['Province/State'],axis = 1)
#recovered_drop_col

In [5]:
# Renaming
confirmed = confirmed.rename(columns={'Country/Region': 'Country'})
recovered = confirmed.rename(columns={'Country/Region': 'Country'})
death = death.rename(columns={'Country/Region': 'Country'})
country = country.rename(columns={'Country_Region': 'Country'})


In [6]:
confirmed_transform = confirmed_drop_cols.transpose()
death_transform = death_drop_cols.transpose()
recovered_transform = recovered_drop_cols.transpose()
#recovered_transform

In [7]:
confirmed_cases = pd.melt(confirmed_drop_cols, id_vars=["Country/Region"], var_name="Dates", value_name= "Confirmed Cases")
death_cases = pd.melt(death_drop_cols, id_vars=["Country/Region"], var_name="Dates", value_name= "Death Cases")
recovered_cases = pd.melt(recovered_drop_cols, id_vars=["Country/Region"], var_name="Dates", value_name= "Recovered Cases")
#confirmed_cases 

In [8]:
covid = pd.merge(confirmed_cases,death_cases)
covid = pd.merge(covid,recovered_cases)
covid['Active Cases'] = covid['Confirmed Cases']- (covid['Death Cases']+covid['Recovered Cases'])
covid['Death Rate (%)'] = (covid['Death Cases']/covid['Confirmed Cases'])*100
covid['Recovery Rate (%)'] = (covid['Recovered Cases']/covid['Confirmed Cases'])*100
#covid

# Total number of cases and number of countries affected

In [9]:
#Displaying total nurmber of confirmed, deaths, recovered and active cases
confirmed_total = int(country['Confirmed'].sum())
deaths_total = int(country['Deaths'].sum())
recovered_total = int(country['Recovered'].sum())
active_total = int(country['Active'].sum())
death_rate = (deaths_total/confirmed_total)*100
recovered_rate = (recovered_total/confirmed_total)*100
countries = country.Country.count()

print("Total number of Confirmed Cases : ", confirmed_total)
print("Total number of Death Cases : ", deaths_total)
print("Total number of Recovered Cases : ", recovered_total)
print("Total number of Active Cases : ", active_total)
print("World Death Rate : ", death_rate)
print("World Recovery Rate : ", recovered_rate)
print("Total number of Countries affected : ", countries)

Total number of Confirmed Cases :  159707593
Total number of Death Cases :  3318264
Total number of Recovered Cases :  95791714
Total number of Active Cases :  28401310
World Death Rate :  2.0777121097805287
World Recovery Rate :  59.97943629392749
Total number of Countries affected :  192


In [10]:
# displaying the total stats

display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: #fff; font-size:30px;'> Confirmed: "  + str(confirmed_total) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(deaths_total) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(recovered_total) + "</span>"+
             "</div>")
       )

In [11]:
sorted_confirmed_country = country.sort_values('Confirmed', ascending = False) 
sorted_death_country = country.sort_values('Deaths', ascending = False) 
sorted_recovered_country = country.sort_values('Recovered', ascending = False)
sorted_active_country = country.sort_values('Active',ascending=False) 
sorted_country = pd.merge(sorted_confirmed_country,sorted_death_country)
sorted_country = pd.merge(sorted_country,sorted_recovered_country)
sorted_country = pd.merge(sorted_country,sorted_active_country)
sorted_country_drop = country.drop(['Last_Update','Lat', 'Long_', 'Incident_Rate', 'People_Tested',
       'People_Hospitalized', 'Mortality_Rate', 'UID', 'ISO3'],axis =1)
sorted_country_drop['Death Rate (%)'] = (sorted_country_drop['Deaths']/sorted_country_drop['Confirmed'])*100
sorted_country_drop['Recovery Rate (%)'] = (sorted_country_drop['Recovered']/sorted_country_drop['Confirmed'])*100
#sorted_country_drop

# Enter the number of countries for which you want the COVID-19 outbreak cases

In [12]:
# sorting the values by confirmed descednding order
# country_df.sort_values('confirmed', ascending= False).head(10).style.background_gradient(cmap='copper')
fig = go.FigureWidget( layout=go.Layout() )
def highlight_col(x):
    r = 'background-color: red'
    y = 'background-color: blue'
    g = 'background-color: green'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = y
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = g
    
    return df1

def show_latest_cases(n):
    n = int(n)
    return country.sort_values('Confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None)

interact(show_latest_cases, n='10')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

# Slide to check the detailed information in each representation

# Bubble chart representation

In [13]:
# # plotting the worst affected countries

def bubble_chart(n):
    fig = px.scatter(sorted_country.head(n), x="Country", y="Confirmed", size="Confirmed", color="Country",
               hover_name="Country", size_max=70)
    fig.update_layout(
    title=str(n) +" Worst affected countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 700
    )
    fig.show()

interact(bubble_chart, n=10)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
#widgets.VBox([fig], layout=ipywLayout)

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

# Choropleth map representation

In [14]:
fig = px.choropleth(country, locations = 'Country', locationmode = 'country names', color = 'Confirmed', animation_frame = 'Last_Update')
fig.update_layout(title_text = 'Countries with Confirmed cases',)
fig.show()
fig = px.choropleth(country, locations = 'Country', locationmode = 'country names', color = 'Deaths', animation_frame = 'Last_Update')
fig.update_layout(title_text = 'Countries with Deaths Reported')
fig.show()
fig = px.choropleth(country, locations = 'Country', locationmode = 'country names', color = 'Recovered', animation_frame = 'Last_Update')
fig.update_layout(title_text = 'Countries with Recovered cases')
fig.show()
fig = px.choropleth(country, locations = 'Country', locationmode = 'country names', color = 'Active', animation_frame = 'Last_Update')
fig.update_layout(title_text = 'Countries with Active cases')
fig.show()

# Bar chart representation

In [15]:
def bar_chart(n):
    fig = px.bar(
    sorted_country.head(n),
    x = "Country",
    y = "Confirmed",
    title= "Top "+ str(n) + " worst affected countries with highest number of confirmed cases", # the axis names
    color_discrete_sequence=["blue"], 
    height=500,
    width=800
)

    fig.show()

interact(bar_chart, n=10)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'

def bar_chart(n):
    fig = px.bar(
    sorted_death_country.head(n),
    x = "Country",
    y = "Deaths",
    title= "Top "+ str(n) + " worst affected countries with highest number of death cases", # the axis names
    color_discrete_sequence=["red"], 
    height=500,
    width=800
)

    fig.show()

interact(bar_chart, n=10)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'


def bar_chart(n):
    fig = px.bar(
    sorted_recovered_country.head(n),
    x = "Country",
    y = "Recovered",
    title= "Top "+ str(n) + " worst affected countries with highest number of recovered cases", # the axis names
    color_discrete_sequence=["green"], 
    height=500,
    width=800
)

    fig.show()

interact(bar_chart, n=10)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'

def bar_chart(n):
    fig = px.bar(
    sorted_active_country.head(n),
    x = "Country",
    y = "Active",
    title= "Top "+ str(n) + " countries with highest number of recovered cases", # the axis names
    color_discrete_sequence=["yellow"], 
    height=500,
    width=800
)

    fig.show()

interact(bar_chart, n=10)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

## Treemap view

In [16]:
fig = px.treemap(sorted_confirmed_country, path = ['Country', 'Confirmed'], values = 'Confirmed',
                title="Top countries with most number of Confirmed cases")
fig.show();

fig = px.treemap(sorted_death_country, path = ['Country', 'Deaths'], values = 'Deaths',
                title="Top countries with most number of Death cases")
fig.show();

fig = px.treemap(sorted_recovered_country, path = ['Country', 'Recovered'], values = 'Recovered',
                title="Top countries with most number of Recovered cases")
fig.show();

fig = px.treemap(sorted_active_country, path = ['Country', 'Active'], values = 'Active',
                title="Top countries with most number of Active cases")
fig.show();

# Pie Chart representation

In [17]:
def pie_chart(n):
    fig = px.pie(sorted_confirmed_country.head(n), values='Confirmed', names='Country',
             title='Confirmed cases (%) across top '+str(n)+' countries',
              hover_data=['Last_Update'], labels={'Last_Update':'Months'})
    fig.update_traces(textposition='inside')
    fig.show()
interact(pie_chart,n=10);

def pie_chart(n):
    fig = px.pie(sorted_death_country.head(n), values='Deaths', names='Country',
             title='Deaths reported (%) across top '+str(n)+' countries',
              hover_data=['Last_Update'], labels={'Last_Update':'Months'})
    fig.update_traces(textposition='inside')
    fig.show()
interact(pie_chart,n=10);

def pie_chart(n):
    fig = px.pie(sorted_recovered_country.head(n), values='Recovered', names='Country',
             title='Recovered cases (%) across top '+str(n)+' countries',
              hover_data=['Last_Update'], labels={'Last_Update':'Months'})
    fig.update_traces(textposition='inside')
    fig.show()
interact(pie_chart,n=10);

def pie_chart(n):
    fig = px.pie(sorted_active_country.head(n), values='Active', names='Country',
             title='Active cases (%) across top '+str(n)+' countries',
              hover_data=['Last_Update'], labels={'Last_Update':'Months'})
    fig.update_traces(textposition='inside')
    fig.show()
interact(pie_chart,n=10);

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…